In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train = pd.read_csv("C:/Users/Ank/Desktop/LoanPredictor/train.csv")
test = pd.read_csv("C:/Users/Ank/Desktop/LoanPredictor/test.csv")
print(train.shape)
print(test.shape)
print(train.head())
print(test.head())

(614, 13)
(367, 12)
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  


In [50]:
print(train.isnull().sum())
print()
print(test.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [51]:
full_data=[train,test]
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [52]:
discrete_var = ['Gender', 'Married', 'Dependents', 'Education','Self_Employed',  'Credit_History', 'Property_Area']
continuous_var = ['ApplicantIncome', 'CoapplicantIncome','Loan_Amount_Term', 'LoanAmount']

In [53]:
for var in discrete_var:
    for data in full_data:
        print(data[var].value_counts())
        print()

Male      489
Female    112
Name: Gender, dtype: int64

Male      286
Female     70
Name: Gender, dtype: int64

Yes    398
No     213
Name: Married, dtype: int64

Yes    233
No     134
Name: Married, dtype: int64

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

0     200
2      59
1      58
3+     40
Name: Dependents, dtype: int64

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

Graduate        283
Not Graduate     84
Name: Education, dtype: int64

No     500
Yes     82
Name: Self_Employed, dtype: int64

No     307
Yes     37
Name: Self_Employed, dtype: int64

1.0    475
0.0     89
Name: Credit_History, dtype: int64

1.0    279
0.0     59
Name: Credit_History, dtype: int64

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

Urban        140
Semiurban    116
Rural        111
Name: Property_Area, dtype: int64



In [54]:
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [55]:
for data in full_data:
    data['Gender'].fillna('Male',inplace=True)
    data['Married'].fillna('Yes',inplace=True)
    data['Dependents'].fillna('0',inplace=True)
    data['Loan_Amount_Term'].fillna(360,inplace=True)
    data['Self_Employed'].fillna('No',inplace=True)
    data['Credit_History'].fillna(1,inplace=True)
    data['LoanAmount'].fillna(data['LoanAmount'].median(),inplace=True)

In [56]:
for data in full_data:
    data['Gender']=data['Gender'].map({'Male':1, 'Female':0})
    data['Married']=data['Married'].map({'Yes':1, 'No':0})
    data['Dependents']=data['Dependents'].map({'0':0,'1':1,'2':2,'3+':3})
    data['Self_Employed']=data['Self_Employed'].map({'Yes':1, 'No':0})
    data['Education']=data['Education'].map({'Graduate':1, 'Not Graduate':0})
    data['Property_Area']=data['Property_Area'].map({'Urban':2, 'Semiurban':1, 'Rural':0})

train['Loan_Status']=train['Loan_Status'].map({'Y':1, 'N':0})

In [57]:
for data in full_data:
    data.loc[(data['ApplicantIncome'] <= 2877.5),'CatApplicantIncome']=0
    data.loc[(data['ApplicantIncome'] > 2877.5)&(data['ApplicantIncome'] <= 3812.5),'CatApplicantIncome']=1
    data.loc[(data['ApplicantIncome'] > 3812.5)&(data['ApplicantIncome'] <= 5795),'CatApplicantIncome']=2
    data.loc[(data['ApplicantIncome'] > 5795),'CatApplicantIncome']=3
    data.loc[(data['CoapplicantIncome'] == 0),'CatCoapplicantIncome']=0
    data.loc[(data['CoapplicantIncome'] > 0)&(data['CoapplicantIncome'] <= 1188.5),'CatCoapplicantIncome']=1
    data.loc[(data['CoapplicantIncome'] > 1188.5)&(data['CoapplicantIncome'] <= 2297.25),'CatCoapplicantIncome']=2
    data.loc[(data['CoapplicantIncome'] > 2297.25),'CatCoapplicantIncome']=3
    data.loc[(data['LoanAmount'] <= 100.25),'CatLoanAmount']=0
    data.loc[(data['LoanAmount'] > 100.25)&(data['LoanAmount'] <= 128),'CatLoanAmount']=1
    data.loc[(data['LoanAmount'] > 128)&(data['LoanAmount'] <= 164.25),'CatLoanAmount']=2
    data.loc[(data['LoanAmount'] > 164.25),'CatLoanAmount']=3
    data.loc[(data['Loan_Amount_Term'] <= 100.25),'CatLoan_Amount_Term']=0
    data.loc[(data['Loan_Amount_Term'] > 100.25)&(data['Loan_Amount_Term'] <= 128),'CatLoan_Amount_Term']=1
    data.loc[(data['Loan_Amount_Term'] > 128)&(data['Loan_Amount_Term'] <= 164.25),'CatLoan_Amount_Term']=2
    data.loc[(data['Loan_Amount_Term'] > 164.25),'CatLoan_Amount_Term']=3

In [58]:
train.corr()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,CatApplicantIncome,CatCoapplicantIncome,CatLoanAmount,CatLoan_Amount_Term
Gender,1.000000,0.364569,0.172914,-0.045364,-0.000525,0.058809,0.082912,0.106904,-0.074030,0.009170,-0.025752,0.017987,0.075344,0.200816,0.143859,-0.000214
Married,0.364569,1.000000,0.334216,-0.012304,0.004489,0.051708,0.075948,0.146546,-0.100912,0.010938,0.004257,0.091478,0.013754,0.276221,0.181072,-0.003976
Dependents,0.172914,0.334216,1.000000,-0.055752,0.056798,0.118202,0.030430,0.163103,-0.103864,-0.040160,-0.000244,0.010118,0.131955,-0.039434,0.114170,-0.039531
Education,-0.045364,-0.012304,-0.055752,1.000000,0.010383,0.140760,0.062290,0.168759,0.073928,0.073658,0.065243,0.085884,0.172585,-0.018593,0.167087,0.008164
Self_Employed,-0.000525,0.004489,0.056798,0.010383,1.000000,0.127180,-0.016100,0.115100,-0.033739,-0.001550,-0.030860,-0.003700,0.183902,-0.083592,0.106133,-0.044627
ApplicantIncome,0.058809,0.051708,0.118202,0.140760,0.127180,1.000000,-0.116605,0.565181,-0.046531,-0.018615,-0.009500,-0.004710,0.530516,-0.230789,0.315482,0.032831
CoapplicantIncome,0.082912,0.075948,0.030430,0.062290,-0.016100,-0.116605,1.000000,0.189218,-0.059383,0.011134,0.010522,-0.059187,-0.186420,0.597941,0.193383,0.018112
LoanAmount,0.106904,0.146546,0.163103,0.168759,0.115100,0.565181,0.189218,1.000000,0.036960,-0.000607,-0.046632,-0.033214,0.454632,0.089192,0.734845,0.060878
Loan_Amount_Term,-0.074030,-0.100912,-0.103864,0.073928,-0.033739,-0.046531,-0.059383,0.036960,1.000000,-0.004705,-0.076120,-0.022549,-0.032542,0.011645,0.029904,0.588053
Credit_History,0.009170,0.010938,-0.040160,0.073658,-0.001550,-0.018615,0.011134,-0.000607,-0.004705,1.000000,0.001963,0.540556,0.043390,-0.006391,0.001885,-0.057410


In [59]:
X_train = train[['Gender', 'Married', 'Dependents', 'Education', 'CatCoapplicantIncome', 'CatLoanAmount',
       'CatLoan_Amount_Term', 'Credit_History', 'Property_Area']]
Y_train = train['Loan_Status']
X_test = test[['Gender', 'Married', 'Dependents', 'Education', 'CatCoapplicantIncome', 'CatLoanAmount',
       'CatLoan_Amount_Term', 'Credit_History', 'Property_Area']]

In [60]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,Y_train)
predictions = lr.predict(X_test)
predictions

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1,

In [61]:
file = open("C:/Users/Ank/Desktop/LoanPredictor/SubmissionLR.csv","w+")
file.write('Loan_ID,Loan_status')
file.write('\n')
for i,p in enumerate(predictions):
    file.write(test['Loan_ID'][i])
    file.write(',')
    if p==1:
        file.write('Y')
    if p==0:
        file.write('N')
    file.write('\n')
file.close()

In [62]:
from sklearn.ensemble import RandomForestClassifier
lr = RandomForestClassifier()
lr.fit(X_train,Y_train)
predictions = lr.predict(X_test)

In [63]:
file = open("C:/Users/Ank/Desktop/LoanPredictor/SubmissionRF.csv","a+")
for i,p in enumerate(predictions):
    file.write(test['Loan_ID'][i])
    file.write(',')
    if p==1:
        file.write('Y')
    if p==0:
        file.write('N')
    file.write('\n')
file.close()

In [64]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lr = LinearDiscriminantAnalysis()
lr.fit(X_train,Y_train)
predictions = lr.predict(X_test)
file = open("C:/Users/Ank/Desktop/LoanPredictor/SubmissionLDA.csv","w+")
for i,p in enumerate(predictions):
    file.write(test['Loan_ID'][i])
    file.write(',')
    if p==1:
        file.write('Y')
    if p==0:
        file.write('N')
    file.write('\n')
file.close()

In [65]:
from sklearn.svm import SVC
lr = SVC()
lr.fit(X_train,Y_train)
predictions = lr.predict(X_test)
file = open("C:/Users/Ank/Desktop/LoanPredictor/SubmissionSVM.csv","w+")
for i,p in enumerate(predictions):
    file.write(test['Loan_ID'][i])
    file.write(',')
    if p==1:
        file.write('Y')
    if p==0:
        file.write('N')
    file.write('\n')
file.close()

In [66]:
from sklearn.naive_bayes import GaussianNB
lr = GaussianNB()
lr.fit(X_train,Y_train)
predictions = lr.predict(X_test)
file = open("C:/Users/Ank/Desktop/LoanPredictor/SubmissionNB.csv","w+")
for i,p in enumerate(predictions):
    file.write(test['Loan_ID'][i])
    file.write(',')
    if p==1:
        file.write('Y')
    if p==0:
        file.write('N')
    file.write('\n')
file.close()